In [2]:
import zarr
import pandas as pd
import numpy as np

In [3]:
in_path = "./teleop_datasets/FeedingJaco-v1.zarr"
out_path = "./teleop_datasets/FeedingJaco-v1.parquet"

In [4]:
store = zarr.open(in_path, mode="r")

In [59]:
pq_dataset = pd.read_parquet("./parquet_datasets/arm_manipulation_test.parquet")

In [60]:
print("~Success rate: ", (1 - pq_dataset.anomaly).sum() / pq_dataset.shape[0])
pq_dataset[pq_dataset.anomaly].head()

~Success rate:  0.2885937870524069


,mdp_state_0,mdp_state_1,mdp_state_2,mdp_state_3,mdp_state_4,mdp_state_5,mdp_state_6,mdp_state_7,mdp_state_8,mdp_state_9,...,mdp_action_11,mdp_action_12,mdp_action_13,sample,anomaly,category,setting,active,action,time
195,0.411666,-0.377213,0.143185,0.700712,0.081952,0.089341,0.703068,0.411666,-0.377213,0.143185,...,-0.986639,-1.000014,0.538746,2,True,1,1,1,0,0.0
196,0.429421,-0.354715,0.202301,0.626628,0.099947,0.127831,0.762238,0.429421,-0.354715,0.202301,...,1.000014,0.469533,1.000014,2,True,1,1,1,0,0.1
197,0.445721,-0.366100,0.230085,0.572972,0.098020,0.108804,0.806385,0.445721,-0.366100,0.230085,...,0.221582,-0.171365,0.988792,2,True,1,1,1,0,0.2
198,0.467493,-0.350416,0.240269,0.523300,0.125156,0.119219,0.834434,0.467493,-0.350416,0.240269,...,0.141679,-0.998215,0.997208,2,True,1,1,1,0,0.3
199,0.481644,-0.335950,0.245992,0.472395,0.145356,0.125546,0.860205,0.481644,-0.335950,0.245992,...,-0.914363,-0.995199,0.998369,2,True,1,1,1,0,0.4


In [58]:
(pq_dataset[~pq_dataset.anomaly].groupby("sample").apply(lambda x: len(x)) == 200).sum()

/tmp/ipykernel_1694544/3767917930.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  (pq_dataset[~pq_dataset.anomaly].groupby("sample").apply(lambda x: len(x)) == 200).sum()


17

In [61]:
pq_dataset.columns

Index(['mdp_state_0', 'mdp_state_1', 'mdp_state_2', 'mdp_state_3',
       'mdp_state_4', 'mdp_state_5', 'mdp_state_6', 'mdp_state_7',
       'mdp_state_8', 'mdp_state_9', 'mdp_state_10', 'mdp_state_11',
       'mdp_state_12', 'mdp_state_13', 'mdp_state_14', 'mdp_state_15',
       'mdp_state_16', 'mdp_state_17', 'mdp_state_18', 'mdp_state_19',
       'mdp_state_20', 'mdp_state_21', 'mdp_state_22', 'mdp_state_23',
       'mdp_state_24', 'mdp_state_25', 'mdp_state_26', 'mdp_state_27',
       'mdp_state_28', 'mdp_state_29', 'mdp_state_30', 'mdp_state_31',
       'mdp_state_32', 'mdp_state_33', 'mdp_state_34', 'mdp_state_35',
       'mdp_state_36', 'mdp_state_37', 'mdp_state_38', 'mdp_state_39',
       'mdp_state_40', 'mdp_state_41', 'mdp_state_42', 'mdp_state_43',
       'mdp_state_44', 'mdp_action_0', 'mdp_action_1', 'mdp_action_2',
       'mdp_action_3', 'mdp_action_4', 'mdp_action_5', 'mdp_action_6',
       'mdp_action_7', 'mdp_action_8', 'mdp_action_9', 'mdp_action_10',
       'mdp_act

In [35]:
N, Ns = np.array(store.data.state).shape
N0, Na = np.array(store.data.action).shape
assert N0 == N

In [51]:
columns = [f"mdp_state_{i}" for i in range(Ns)] + [f"mdp_action_{i}" for i in range(Na)]
df = pd.DataFrame(
    data=np.concatenate([np.array(store.data.state), np.array(store.data.action)], axis=1),
    columns=columns,
)

samples_col = np.zeros(N)
time_col = np.zeros(N)
prev_epi_end = 0
for sample_id, epi_end in enumerate(np.array(store.meta.episode_ends)):
    samples_col[prev_epi_end: epi_end] = sample_id
    time_col[prev_epi_end: epi_end] = np.arange(epi_end - prev_epi_end) / 10 #hz
    prev_epi_end = epi_end

df["sample"] = samples_col.astype(np.int_)
df["anomaly"] = True
df["category"] = 0
df["setting"] = 0
df["active"] = 0
df["time"] = time_col

In [61]:
df.to_parquet(out_path)

In [62]:
df

,mdp_state_0,mdp_state_1,mdp_state_2,mdp_state_3,mdp_state_4,mdp_state_5,mdp_state_6,mdp_state_7,mdp_state_8,mdp_state_9,...,mdp_action_3,mdp_action_4,mdp_action_5,mdp_action_6,sample,anomaly,category,setting,active,time
0,0.214849,0.231885,0.796235,0.473767,0.474204,0.524861,0.524591,0.460681,-0.140167,-0.095949,...,2.098135e-08,9.508962e-07,0.000001,-6.678583e-08,0,True,0,0,0,0.0
1,0.214855,0.231891,0.796201,0.473726,0.474184,0.524871,0.524638,0.455601,-0.143481,-0.092114,...,4.981664e-04,-1.693518e-04,0.000230,-6.906188e-04,0,True,0,0,0,0.1
2,0.214865,0.231893,0.796200,0.473729,0.474186,0.524861,0.524642,0.456459,-0.140056,-0.094324,...,3.979625e-04,-1.938228e-04,0.000287,-5.373867e-04,0,True,0,0,0,0.2
3,0.214873,0.231896,0.796185,0.473723,0.474184,0.524859,0.524652,0.455734,-0.143078,-0.092396,...,-4.720549e-02,8.642333e-02,0.146594,-4.193262e-02,0,True,0,0,0,0.3
4,0.215027,0.234733,0.796232,0.473866,0.474063,0.524708,0.524782,0.456742,-0.136863,-0.094531,...,-7.666107e-02,1.465842e-01,0.249005,-6.765395e-02,0,True,0,0,0,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21256,-0.171954,0.346950,0.787937,0.474601,0.473925,0.525525,0.523424,0.060796,0.010456,0.000441,...,1.180232e-01,-1.634298e-01,0.016827,-4.706968e-01,249,True,0,0,0,4.7
21257,-0.179048,0.345453,0.787629,0.474397,0.473973,0.525257,0.523833,0.053702,0.008960,0.000133,...,1.609284e-01,-1.609439e-01,0.082144,-4.999744e-01,249,True,0,0,0,4.8
21258,-0.187027,0.344438,0.787402,0.474248,0.474009,0.525059,0.524135,0.045722,0.007944,-0.000094,...,1.986355e-01,-1.747361e-01,0.143886,-5.249348e-01,249,True,0,0,0,4.9
21259,-0.195574,0.343777,0.787218,0.474110,0.473944,0.524943,0.524434,0.037176,0.007283,-0.000278,...,1.406226e-01,-1.231623e-01,0.102294,-3.745734e-01,249,True,0,0,0,5.0
